<a href="https://colab.research.google.com/github/JstnClmnt/NLP-Text-Summarization/blob/practice/NLP_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import re
np.random.seed(1)

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import os
 
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = dirName+"/"+entry
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles        

In [0]:
!unzip "/content/drive/Team Drives/AI Lords/NLP-Text-Summarization/dataset.zip"

Archive:  /content/drive/Team Drives/AI Lords/NLP-Text-Summarization/dataset.zip
replace data/business/001.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/business/001.txt   
  inflating: data/business/002.txt   
  inflating: data/business/003.txt   
  inflating: data/business/004.txt   
  inflating: data/business/005.txt   
  inflating: data/business/006.txt   
  inflating: data/business/007.txt   
  inflating: data/business/008.txt   
  inflating: data/business/009.txt   
  inflating: data/business/010.txt   
  inflating: data/business/011.txt   
  inflating: data/business/012.txt   
  inflating: data/business/013.txt   
  inflating: data/business/014.txt   
  inflating: data/business/015.txt   
  inflating: data/business/016.txt   
  inflating: data/business/017.txt   
  inflating: data/business/018.txt   
  inflating: data/business/019.txt   
  inflating: data/business/020.txt   
  inflating: data/business/021.txt   
  inflating: data/business/022.txt   
  inflating:

In [3]:
import pandas as pd


dirName = 'data';
    
# Get the list of all files in directory tree at given path
listOfFiles = getListOfFiles(dirName)
    

df=pd.DataFrame(columns=["Title","Description","Category"])
# Print the files    
for elem in listOfFiles:
    print("Reading: "+elem)
    file1 = open(elem,"r",encoding="latin-1") 
    sampleNews=file1.read().split("\n")
    newsDesc=""
    for strline in sampleNews[1:len(sampleNews)]:
        newsDesc+=strline
    """
    print("Category: "+elem.split("/")[1])
    print("Title:"+sampleNews[0])
    print("Description:"+newsDesc[0:50])
    """
    dfsample=pd.DataFrame(columns=["Title","Description","Category"],data=[[sampleNews[0],newsDesc,elem.split("/")[1]]])
    df=df.append(dfsample)

Reading: data/politics/173.txt
Reading: data/politics/247.txt
Reading: data/politics/204.txt
Reading: data/politics/311.txt
Reading: data/politics/098.txt
Reading: data/politics/262.txt
Reading: data/politics/241.txt
Reading: data/politics/382.txt
Reading: data/politics/201.txt
Reading: data/politics/049.txt
Reading: data/politics/088.txt
Reading: data/politics/357.txt
Reading: data/politics/341.txt
Reading: data/politics/269.txt
Reading: data/politics/303.txt
Reading: data/politics/081.txt
Reading: data/politics/070.txt
Reading: data/politics/157.txt
Reading: data/politics/367.txt
Reading: data/politics/035.txt
Reading: data/politics/033.txt
Reading: data/politics/392.txt
Reading: data/politics/143.txt
Reading: data/politics/288.txt
Reading: data/politics/246.txt
Reading: data/politics/350.txt
Reading: data/politics/238.txt
Reading: data/politics/129.txt
Reading: data/politics/361.txt
Reading: data/politics/004.txt
Reading: data/politics/351.txt
Reading: data/politics/236.txt
Reading:

In [4]:
df=df.reset_index()
df=df.drop(list(df)[0], axis=1)
df["Description"].head()[0]

'The Scottish Executive has lost an appeal against an inmate\'s compensation for being forced to slop out in prison.Armed robber Robert Napier, 25, won Â£2,450 after he claimed he suffered an outbreak of the skin complaint, eczema, when slopping out at Barlinnie Prison. Napier said that the practice, where prisoners use buckets in their cells as toilets, breached his human rights. On Thursday, the Court of Session threw out a move by the executive to apply a more rigorous standard of proof.The executive faces more than 1,000 similar claims for damages from prisoners and former inmates. More than 310 actions have already been raised in the Court of Session and sheriff courts in Scotland. An executive spokesman said: "We will study this judgement in detail. Much has changed to address the issues raised in the Napier case, for example, slopping out has ended at Barlinnie and work in other prisons is being accelerated. "Today\'s judgement does not affect the outcome of other cases." Napier

In [5]:
df=df[["Title","Description"]]
df.head()

,Title,Description
0,Ministers lose slopping out case,The Scottish Executive has lost an appeal agai...
1,Kennedy to make temple address,"Charles Kennedy is set to address 2,000 people..."
2,Correction agency plans dropped,Plans to create a single correctional agency f...
3,Talks aim to avert pension strike,Talks aimed at averting a series of national s...
4,Ministers deny care sums 'wrong',Ministers have insisted they are committed to ...


In [0]:
df['Title'] = df['Title'].str.replace("[^a-zA-Z#]"," ")
df['Description'] = df['Description'].str.replace("[^a-zA-Z#]"," ")
df=df.dropna()

In [0]:
df=df.sample(n=1500)

In [8]:
from keras.preprocessing.text import Tokenizer
VOCAB_SIZE = 10000
tokenizer = Tokenizer(oov_token=1,num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(df["Description"])
article_sequences = tokenizer.texts_to_sequences(df["Description"])
art_word_index = tokenizer.word_index
len(art_word_index)


Using TensorFlow backend.


23627

In [9]:
art_word_index_1500 = {}
counter = 0
for word in art_word_index.keys():
    if art_word_index[word] == 0:
        print("found 0!")
        break
    if art_word_index[word] > VOCAB_SIZE:
        continue
    else:
        art_word_index_1500[word] = art_word_index[word]
        counter += 1
print(counter)

10000


In [10]:
tokenizer.fit_on_texts(df["Title"])
summary_sequences = tokenizer.texts_to_sequences(df["Title"])
sum_word_index = tokenizer.word_index
len(sum_word_index)

23776

In [11]:
sum_word_index_1500 = {}
counter = 0
for word in sum_word_index.keys():
    if sum_word_index[word] == 0:
        print("found 0!")
        break
    if sum_word_index[word] > VOCAB_SIZE:
        continue
    else:
        sum_word_index_1500[word] = sum_word_index[word]
        counter += 1
print(counter)

10000


In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(article_sequences, summary_sequences, test_size=0.20, random_state=42)

print(X_train[0])

[51, 958, 487, 5979, 21, 39, 865, 3, 4566, 5453, 55, 1, 317, 944, 5054, 15, 452, 3, 2310, 2415, 28, 3659, 7199, 93, 1, 2, 487, 45, 865, 3, 383, 577, 465, 770, 8, 146, 7, 2570, 5, 21, 2715, 463, 8, 1549, 764, 5054, 62, 3426, 750, 42, 606, 1263, 156, 14, 34, 5979, 234, 47, 117, 16, 10, 80, 1, 7, 2310, 2, 8634, 4, 2, 487, 328, 6, 577, 1, 118, 13, 465, 463, 8, 3251, 20, 39, 1, 9, 876, 143, 47, 36, 6, 2705, 9, 2, 146, 7, 1549, 25, 89, 4625, 3, 4948, 40, 2705, 1, 78, 30, 58, 1773, 2, 52, 764, 2, 487, 8, 234, 14, 11, 30, 38, 1339, 40, 1766, 5013, 13, 6, 190, 3, 190, 1720, 205, 13, 3261, 1708, 3, 5054, 1576, 32, 60, 158, 944, 3, 94, 256, 5, 1443, 32, 60, 158, 3, 156, 2, 675, 912, 5877, 32, 23, 87, 78, 19, 875, 19, 499, 16, 14, 2, 487, 45, 142, 3791, 30, 38, 17, 277, 3, 91, 40, 454, 764, 117, 2, 234, 5979, 328, 1, 1879, 4, 40, 2095, 1250, 121, 5607, 27, 291, 39, 163, 1804, 52, 592, 5, 2, 49, 27, 707, 3, 2, 2570, 5046, 937, 13, 465, 2, 487, 36, 2265, 39, 2631, 3, 156, 171, 5013, 7, 2, 73, 19, 15

In [0]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

--2019-05-19 17:00:57--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-05-19 17:00:57--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  11.6MB/s    in 53s     

2019-05-19 17:01:49 (15.7 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [0]:
!unzip "glove.6B.zip"

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
import numpy as np

embeddings_index = dict()
f = open('glove.6B.300d.txt',encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [14]:
EMB_DIM=300
num_words=VOCAB_SIZE+1
print("Number of Words:"+str(num_words))

#MAX_LENGTH = len(max(X_train, key=len))
#print("Max Length: "+str(MAX_LENGTH))  # 271

MAX_LENGTH=75

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
train_sentences_X = pad_sequences(X_train, maxlen=MAX_LENGTH, padding='post',truncating="post")
test_sentences_X = pad_sequences(X_test, maxlen=MAX_LENGTH, padding='post',truncating="post")
train_sentences_Y= pad_sequences(y_train, maxlen=MAX_LENGTH, padding='post',truncating="post")
test_sentences_Y= pad_sequences(y_test, maxlen=MAX_LENGTH, padding='post',truncating="post")
#y_train=to_categorical(y_train)
#y_test=to_categorical(y_test)
print(train_sentences_X[0])
print(train_sentences_Y[0])

Number of Words:10001
[  51  958  487 5979   21   39  865    3 4566 5453   55    1  317  944
 5054   15  452    3 2310 2415   28 3659 7199   93    1    2  487   45
  865    3  383  577  465  770    8  146    7 2570    5   21 2715  463
    8 1549  764 5054   62 3426  750   42  606 1263  156   14   34 5979
  234   47  117   16   10   80    1    7 2310    2 8634    4    2  487
  328    6  577    1  118]
[5212 5158 8769   24 6827    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]


In [15]:
embedding_matrix_art=np.zeros((num_words,EMB_DIM))
numNoEmb=0
#print(word2index)
for word,i in art_word_index_1500.items():
    embedding_vector=embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix_art[i]=embedding_vector
    else:
      numNoEmb+=1
print(numNoEmb)

55


In [16]:
embedding_matrix_sum=np.zeros((num_words,EMB_DIM))
#print(word2index)
numNoEmb=0
for word,i in sum_word_index_1500.items():
    if i>num_words:
        continue
    embedding_vector=embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix_sum[i]=embedding_vector
    else:
      numNoEmb+=1
print(numNoEmb)

54


In [17]:
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import concatenate,Input,Dense, CuDNNLSTM,LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation,Dropout,Add
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.keras import regularizers
from tensorflow.keras.initializers import Constant
import numpy as np
from tensorflow.keras.utils import plot_model

HIDDEN_UNITS=512

"""
Chatbot Inspired Encoder-Decoder-seq2seq
"""
encoder_inputs = Input(shape=(MAX_LENGTH, ))
encoder_embedding = Embedding(num_words,EMB_DIM,embeddings_initializer=Constant(embedding_matrix_art),input_length=MAX_LENGTH,trainable=True,mask_zero=True)(encoder_inputs)
encoder_LSTM = LSTM(HIDDEN_UNITS, return_state=True)
encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)

decoder_inputs = Input(shape=(MAX_LENGTH, ))
decoder_embedding = Embedding(num_words,EMB_DIM,embeddings_initializer=Constant(embedding_matrix_sum),input_length=MAX_LENGTH,trainable=True,mask_zero=True)(decoder_inputs)
decoder_LSTM = LSTM(HIDDEN_UNITS, return_state=True, return_sequences=True)
decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=[state_h, state_c])

# dense_layer = Dense(VOCAB_SIZE, activation='softmax')
outputs = TimeDistributed(Dense(num_words, activation='softmax'))(decoder_outputs)
model = tf.keras.Model([encoder_inputs, decoder_inputs], outputs)


model.compile(optimizer=Adam(0.001), loss="categorical_crossentropy", metrics=["accuracy"])

print(model.summary())
plot_model(model, to_file='model_seq2seq.png')

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 75)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 75)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 75, 300)      3000300     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 75, 300)      3000300     input_2[0][0]                    
_____________________________________

In [18]:
num_samples_train = len(train_sentences_Y)
print(num_samples_train)
decoder_output_data_train = np.zeros(shape=(num_samples_train, MAX_LENGTH, num_words))

1200


In [19]:
num_samples_test = len(test_sentences_Y)
print(num_samples_test)
decoder_output_data_test = np.zeros(shape=(num_samples_test, MAX_LENGTH, num_words))

300


In [0]:
for i, seqs in enumerate(train_sentences_X):
    for j, seq in enumerate(seqs):
        if j > 0:
            decoder_output_data_train[i][j][seq] = 1
            
for i, seqs in enumerate(test_sentences_X):
    for j, seq in enumerate(seqs):
        if j > 0:
            decoder_output_data_test[i][j][seq] = 1

In [21]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

save = ModelCheckpoint('best_model.hdf5', save_best_only=True, monitor='acc', mode='max')
history = model.fit([train_sentences_X, train_sentences_Y], 
                     decoder_output_data_train, 
                     epochs=100, 
                     batch_size=50,
                     validation_data=([test_sentences_X, test_sentences_Y], decoder_output_data_test),
                     callbacks=[save])

Train on 1200 samples, validate on 300 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
1200/1200 [==============================] - 27s 22ms/sample - loss: 6.4840 - acc: 0.0313 - val_loss: 6.0296 - val_acc: 0.0332
Epoch 2/100
1200/1200 [==============================] - 25s 21ms/sample - loss: 5.4621 - acc: 0.0361 - val_loss: 6.0878 - val_acc: 0.0357
Epoch 3/100
1200/1200 [==============================] - 16s 14ms/sample - loss: 5.3324 - acc: 0.0350 - val_loss: 6.1752 - val_acc: 0.0332
Epoch 4/100
1200/1200 [==============================] - 24s 20ms/sample - loss: 5.2801 - acc: 0.0362 - val_loss: 6.2046 - val_acc: 0.0363
Epoch 5/100
1200/1200 [==============================] - 24s 20ms/sample - loss: 5.2352 - acc: 0.0405 - val_loss: 6.1798 - val_acc: 0.0407
Epoch 6/100
1200/1200 [==============================] - 17s 14ms/sample - loss: 5.1582 - acc: 0.0418 - val_loss: 6.1560 - val_acc: 0.0401
Epoch 7/100
1200/1200 [==============================] - 16s 13ms/sampl